In [ ]:
import sys
from shutil import copyfile

copyfile(src = "../input/shopee-utils/utils.py", dst = "../working/utils.py")
sys.path.append("../input/timm-pytorch-image-models/pytorch-image-models-master")

In [ ]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.nn import functional as F
from torchvision import transforms
from utils import ShopeeTrainDataset, ShopeeImageDataset, ShopeeTextTrainDataset, ShopeeTextDataset
from utils import get_metric, validate

from transformers import AutoTokenizer, AutoModel
import timm

import cudf
import cuml

import os
from tqdm import tqdm
import math

In [ ]:
class config:
    PATH = "../input/shopee-product-matching/"
    
    image_model_name = "eca_nfnet_l0"
    image_model_path = "../input/shopeemodel/eca_nfnet_l0_flexibleMargin_epoch_8.pt"
    text_model_name = "distilbert-base-multilingual-cased"
    text_model_path = "../input/shopeemodel/distilbert-base-multilingual-cased_epoch_6.pt"
    
    n_classes = 9024
    batch_size = 16

In [ ]:
class ArcFace(nn.Module):
    """ NN module for projecting extracted embeddings onto the sphere surface """
    
    def __init__(self, in_features, out_features, s=30, m=0.5):
        super(ArcFace, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.s = s
        self.m = m
        self.cos_m = math.cos(self.m)
        self.sin_m = math.sin(self.m)
        self.arc_min = math.cos(math.pi - self.m)
        self.margin_min = math.sin(math.pi - self.m) * self.m
        self.weight = nn.Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)
    
    def _update_margin(self, new_margin):
        self.m = new_margin
        self.cos_m = math.cos(self.m)
        self.sin_m = math.sin(self.m)
        self.arc_min = math.cos(math.pi - self.m)
        self.margin_min = math.sin(math.pi - self.m) * self.m

    def forward(self, embedding, label):
        cos = F.linear(F.normalize(embedding), F.normalize(self.weight))
        sin = torch.sqrt(1.0 - torch.pow(cos, 2)).clamp(0, 1)
        phi = cos * self.cos_m - sin * self.sin_m
        phi = torch.where(cos > self.arc_min, phi, cos - self.margin_min)

        one_hot = torch.zeros(cos.size(), device=device)
        one_hot.scatter_(1, label.view(-1, 1).long(), 1)
        logits = one_hot * phi + (1.0 - one_hot) * cos
        logits *= self.s
        return logits

In [ ]:
class Model(nn.Module):
    def __init__(self, model_name, n_classes, margin=0.5, fc_dim=1024):
        super(Model, self).__init__()
        print("Building Model Backbone for {} model".format(model_name))
        self.model_name = model_name
        self.backbone = timm.create_model(model_name, pretrained=True)
        
        if "resnet" in model_name:
            feat_size = self.backbone.fc.in_features
            self.backbone.fc = nn.Identity()
            self.backbone.global_pool = nn.Identity()
        
        elif "eca_nfnet" in model_name:
            feat_size = self.backbone.head.fc.in_features
            self.backbone.head.fc = nn.Identity()
            self.backbone.head.global_pool = nn.Identity()
                
        elif "efficientnet" in model_name:
            feat_size = self.backbone.classifier.in_features
            self.backbone.classifier = nn.Identity()
            self.backbone.global_pool = nn.Identity()
        
        self.pooling =  nn.AdaptiveAvgPool2d(1)
        self.dropout = nn.Dropout(p=0.1)
        self.fc = nn.Linear(feat_size, fc_dim)
        self.bn = nn.BatchNorm1d(fc_dim)
        self.margin = ArcFace(fc_dim, n_classes, m=margin)
        self._init_params()

    def _init_params(self):
        nn.init.xavier_normal_(self.fc.weight)
        nn.init.constant_(self.fc.bias, 0)
        nn.init.constant_(self.bn.weight, 1)
        nn.init.constant_(self.bn.bias, 0)

    def forward(self, x, labels=None):
        batch_size = x.shape[0]
        x = self.backbone(x)
        x = self.pooling(x).view(batch_size, -1)
        
        x = self.dropout(x)
        x = self.fc(x)
        x = self.bn(x)
        x = F.normalize(x,dim=1)
        if labels is not None:
            return self.margin(x,labels)
        else:
            return x

In [ ]:
class TextModel(nn.Module):
    def __init__(self, model_name, n_classes, margin=0.5, fc_dim=1024):
        super(TextModel, self).__init__()
        print("Building Model Backbone for {} model".format(model_name))
        self.model_name = model_name
        self.tokenizer = AutoTokenizer.from_pretrained("{model_name}".format(model_name=model_name), TOKENIZERS_PARALLELISM=False)
        self.backbone = AutoModel.from_pretrained("{model_name}".format(model_name=model_name))
        self.feat_size = self.backbone.config.hidden_size

        self.pooling =  nn.AdaptiveAvgPool2d(1)
        self.dropout = nn.Dropout(p=0.1)
        self.fc = nn.Linear(self.feat_size, fc_dim)
        self.bn = nn.BatchNorm1d(fc_dim)
        self.margin = ArcFace(fc_dim, n_classes, m=margin)
        self._init_params()

    def _init_params(self):
        nn.init.xavier_normal_(self.fc.weight)
        nn.init.constant_(self.fc.bias, 0)
        nn.init.constant_(self.bn.weight, 1)
        nn.init.constant_(self.bn.bias, 0)

    def forward(self, text, labels=None):
        inputs = self.tokenizer(text, truncation=True, padding=True, return_tensors="pt")
        output = self.backbone(input_ids = inputs["input_ids"].to(device), attention_mask = inputs["attention_mask"].to(device))
        embedding = output[0][:, 0, :] 
        x = self.dropout(embedding)
        x = self.fc(x)
        x = self.bn(x)
        x = F.normalize(x,dim=1)
        if labels is not None:
            return self.margin(x,labels)
        else:
            return x

In [ ]:
def read_dataset(name="train"):
    assert name in {"train", "test"}
    df = pd.read_csv(config.PATH + '{}.csv'.format(name))
    df["image_path"] = config.PATH + '{}_images/'.format(name) + df['image']

    return df

In [ ]:
df = read_dataset("train")
label_group_dict = df.groupby("label_group").posting_id.agg("unique").to_dict()
df['target'] = df.label_group.map(label_group_dict)

df.head()

In [ ]:
n_classes = len(df["label_group"].unique())
num = int(0.2 * n_classes)
np.random.seed(1)
test_group = np.random.choice(df["label_group"].unique(), num)
#test_group
df_test = df[df["label_group"].isin(test_group)]
df_train = df[~df["label_group"].isin(test_group)]

if torch.cuda.is_available():
    df_cu = cudf.DataFrame(df) 
    df_train_cu = cudf.DataFrame(df_train) 
    df_test_cu = cudf.DataFrame(df_test) 

print(len(df_train), len(df_test))

In [ ]:
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

test_dataset = ShopeeImageDataset(df_test, transform = transform)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=config.batch_size, shuffle=False, num_workers=2)
df_text_dataset = ShopeeTextDataset(df_test)
df_text_dataloader = torch.utils.data.DataLoader(df_text_dataset, batch_size=config.batch_size, shuffle=False, num_workers=2)

whole_dataset = ShopeeImageDataset(df, transform = transform)
whole_dataloader = torch.utils.data.DataLoader(whole_dataset, batch_size=config.batch_size, shuffle=False, num_workers=2)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')

def get_model(model_name, model_path, n_classes):
    model = Model(model_name, n_classes)
    if model_path is not None:
        model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()
    return model.to(device)

def get_text_model(model_name, model_path, n_classes):
    model = TextModel(model_name, n_classes)
    if model_path is not None:
        model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()
    return model.to(device)

In [ ]:
def get_image_feature(model, dataloader):
    image_features = []
    with torch.no_grad():
        for images in tqdm(dataloader):
            images = images.to(device)
            features = model(images)
            image_features.append(features)
            del images
    image_features = torch.cat(image_features, axis=0)

    torch.cuda.empty_cache()   
    return image_features


def get_tfidf_feature(df, max_features):
    if torch.cuda.is_available():
        from cuml.feature_extraction.text import TfidfVectorizer
    else:
        from sklearn.feature_extraction.text import TfidfVectorizer
    
    model = TfidfVectorizer(stop_words='english', max_features=max_features)
    model.fit(df.title)

    tfidf_features = model.transform(df.title).toarray()
    tfidf_features = torch.Tensor(tfidf_features).to(device)
    return tfidf_features

def get_bert_feature(model, dataloader):
    text_features = []
    with torch.no_grad():
        for text in tqdm(dataloader):
            text = list(text)
            features = model(text)
            text_features.append(features)
            del text
    text_features = torch.cat(text_features, axis=0)

    torch.cuda.empty_cache()   
    return text_features

PRETRAIN

In [ ]:
model_name = "resnet34"
model = get_model(model_name, None, config.n_classes)

image_features = get_image_feature(model, test_dataloader)

thresholds = np.arange(0.6, 0.95, 0.05)
# least_thresholds = np.arange(0.2, 0.6, 0.1)
# least_thresholds = np.arange(0.4, 0.5, 0.1)
for threshold in thresholds:
#     for least_threshold in least_thresholds:
    f1, prec, rec = validate(image_features, df_test, threshold)
    print("Threshold: {:2f} F1: {:5f} Precision: {:5f} Recall: {:5f}".format(threshold, f1, prec, rec))

In [ ]:
model_name = "bert-base-multilingual-cased"
text_model = get_text_model(model_name, None, config.n_classes)
bert_features = get_bert_feature(text_model, df_text_dataloader)

thresholds = np.arange(0.6, 0.95, 0.05)
# least_thresholds = np.arange(0.2, 0.6, 0.1)
# least_thresholds = np.arange(0.4, 0.5, 0.1)
for threshold in thresholds:
#     for least_threshold in least_thresholds:
    f1, prec, rec = validate(bert_features, df_test, threshold)
    print("Threshold: {:2f}  F1: {:5f} Precision: {:5f} Recall: {:5f}".format(threshold, f1, prec, rec))

In [ ]:
model = get_model(config.image_model_name, config.image_model_path, config.n_classes)
image_features = get_image_feature(model, test_dataloader)

# thresholds = np.arange(0.6, 0.95, 0.05)
# least_thresholds = np.arange(0.2, 0.6, 0.1)
# least_thresholds = np.arange(0.4, 0.5, 0.1)
# for threshold in thresholds:
#     for least_threshold in least_thresholds:
#     f1, prec, rec = validate(image_features, df_test, threshold)
#     print("Threshold: {:2f} F1: {:5f} Precision: {:5f} Recall: {:5f}".format(threshold, f1, prec, rec))

IMAGE_THRESHOLD = 0.8
IMAGE_LEAST_THRESHOLD = 0.6

f1, prec, rec = validate(image_features, df_test, IMAGE_THRESHOLD, least_threshold=IMAGE_LEAST_THRESHOLD)
print("Threshold: {:2f} F1: {:5f} Precision: {:5f} Recall: {:5f}".format(IMAGE_THRESHOLD, f1, prec, rec))

In [ ]:
# image_features = get_image_feature(model, whole_dataloader)

# thresholds = np.arange(0.4, 0.9,0.05)
# for threshold in thresholds:
#     f1, prec, rec = validate(image_features, threshold, df)
#     print("Threshold: {:2f} F1: {:5f} Precision: {:5f} Recall: {:5f}".format(threshold, f1, prec, rec))

In [ ]:
text_model = get_text_model(config.text_model_name, config.text_model_path, config.n_classes)
bert_features = get_bert_feature(text_model, df_text_dataloader)

# thresholds = np.arange(0.6, 0.95, 0.05)
# # least_thresholds = np.arange(0.2, 0.6, 0.1)
# # least_thresholds = np.arange(0.4, 0.5, 0.1)
# for threshold in thresholds:
# #     for least_threshold in least_thresholds:
#     f1, prec, rec = validate(bert_features, df_test, threshold)
#     print("Threshold: {:2f}  F1: {:5f} Precision: {:5f} Recall: {:5f}".format(threshold, f1, prec, rec))

BERT_THRESHOLD = 0.85
BERT_LEAST_THRESHOLD = 0.65

f1, prec, rec = validate(bert_features, df_test, BERT_THRESHOLD, least_threshold=BERT_LEAST_THRESHOLD)
print("Threshold: {:2f}  F1: {:5f} Precision: {:5f} Recall: {:5f}".format(BERT_THRESHOLD, f1, prec, rec))

In [ ]:
concat_features = torch.cat([image_features, bert_features], axis=1)

# thresholds = np.arange(0.6, 0.95, 0.05)
# # least_thresholds = np.arange(0.2, 0.6, 0.1)
# # least_thresholds = np.arange(0.3, 0.35, 0.1)
# for threshold in thresholds:
# #     for least_threshold in least_thresholds:
#         f1, prec, rec = validate(concat_features, df_test, threshold)
#         print("Threshold: {:2f} F1: {:5f} Precision: {:5f} Recall: {:5f}".format(threshold, f1, prec, rec))

CONCAT_THRESHOLD = 0.65
CONCAT_LEAST_THRESHOLD = 0.45

f1, prec, rec = validate(concat_features, df_test, CONCAT_THRESHOLD, least_threshold=CONCAT_LEAST_THRESHOLD)
print("Threshold: {:2f}  F1: {:5f} Precision: {:5f} Recall: {:5f}".format(CONCAT_THRESHOLD, f1, prec, rec))

In [ ]:
TFIDF_FEATURES = 25000
tfidf_features = get_tfidf_feature(df_cu, TFIDF_FEATURES)
thresholds = np.arange(0.6, 0.95, 0.05)
for threshold in thresholds:
    f1, prec, rec = validate(tfidf_features, df, threshold)
    print("Threshold: {:2f} F1: {:5f} Precision: {:5f} Recall: {:5f}".format(threshold, f1, prec, rec))

In [ ]:
from utils import DistancePredict
from functools import reduce

def union(x,y):
    return np.union1d(x,y)

def intersect(x,y):
    return np.intersect1d(x,y)

# df["pred"] = df.apply(lambda row: union(row['image_pred'], row["tfidf_pred"]),axis=1)
# df["wait"] = df.apply(lambda row: intersect(row['image_pred_wait'], row["text_pred_wait"]),axis=1)
# df["pred"] = df.apply(lambda row: union(row['pred'], row["wait"]),axis=1)

df_test["image_pred"] = DistancePredict(image_features, df_test, threshold= IMAGE_THRESHOLD, least_threshold=IMAGE_LEAST_THRESHOLD)
df_test["text_pred"] = DistancePredict(bert_features, df_test, threshold= BERT_THRESHOLD, least_threshold=BERT_LEAST_THRESHOLD)
df_test["concat_pred"] = DistancePredict(concat_features, df_test, threshold= CONCAT_THRESHOLD, least_threshold=CONCAT_LEAST_THRESHOLD)

df_test["pred"] = df_test.apply(lambda row: reduce(union, [row['image_pred'], row["text_pred"], row["concat_pred"]]),axis=1)

f1, prec, rec = get_metric(df_test["target"], df_test["pred"])
print("Mean F1: {:f}".format(f1))
print("Mean Precision: {:f}".format(prec))
print("Mean Recall: {:f}".format(rec))